### Installation

In [ ]:
!pip install psycopg2
!pip install python-dotenv
!pip install openpyxl
!pip install SQLAlchemy

!sudo apt-get update
!sudo apt-get install openvpn -y

### Load credentials and Variables

In [61]:
import psycopg2
from psycopg2 import sql
import json

# Load environment variables
import os
from dotenv import load_dotenv

load_dotenv()

# Get database credentials from environment variables
dbname = os.getenv("dbname")
engine = os.getenv("engine")
host = os.getenv("host")
password = os.getenv("password")
port = os.getenv("port")
username = os.getenv("username")
vpn_path = os.getenv("vpn_path")
api_key = os.getenv("api_key")

# Database connection parameters
db_params = {
    'dbname': dbname,
    'user': username,
    'password': password,
    'host': host,
    'port': port  # Add port if necessary
}


# Load the configuration file
with open('config.json', 'r') as file:
    config = json.load(file)

# Extract schema from the config
schema = config['schema']

# Combine the lines to form the full prompt with f-string
prompt_template = "\n".join(config["prompt"])

# Format the prompt with the schema variable
base_prompt = prompt_template.format(schema=schema)


import subprocess

# Step 3: Start OpenVPN in the background and check the status
openvpn_process = subprocess.Popen(['sudo', 'openvpn', '--config', vpn_path],
                                   stdout=subprocess.PIPE,
                                   stderr=subprocess.PIPE)

### Connection Troubleshooting

In [ ]:
import subprocess
import time

# Replace '/path/to/your/file.ovpn' with the actual path to your .ovpn file
openvpn_process = subprocess.Popen(['sudo', 'openvpn', '--config', vpn_path],
                                   stdout=subprocess.PIPE,
                                   stderr=subprocess.PIPE)

# Collect logs for debugging
vpn_connected = False
for _ in range(30):  # Try for up to 30 seconds
    output = openvpn_process.stdout.readline().decode('utf-8')
    error_output = openvpn_process.stderr.readline().decode('utf-8')
    if "Initialization Sequence Completed" in output:
        vpn_connected = True
        break
    if error_output:
        print(f"Error: {error_output.strip()}")
    time.sleep(1)

if vpn_connected:
    print("VPN connection established successfully.")
else:
    print("Failed to establish VPN connection.")
    openvpn_process.terminate()
    raise RuntimeError("VPN connection failed")


In [ ]:
import subprocess
import time
import psycopg2

# Step 1: Install OpenVPN and dependencies
#!sudo apt-get update
#!sudo apt-get install -y openvpn easy-rsa

# Step 2: Remove existing TUN/TAP interfaces
try:
    subprocess.check_output(['sudo', 'ip', 'link', 'delete', 'tun0'])
except subprocess.CalledProcessError:
    pass
try:
    subprocess.check_output(['sudo', 'ip', 'link', 'delete', 'tun1'])
except subprocess.CalledProcessError:
    pass
try:
    subprocess.check_output(['sudo', 'ip', 'link', 'delete', 'tun2'])
except subprocess.CalledProcessError:
    pass
try:
    subprocess.check_output(['sudo', 'ip', 'link', 'delete', 'tun3'])
except subprocess.CalledProcessError:
    pass

# Step 3: Start OpenVPN in the background and check the status
openvpn_process = subprocess.Popen(['sudo', 'openvpn', '--config', vpn_path],
                                   stdout=subprocess.PIPE,
                                   stderr=subprocess.PIPE)

# Collect logs for debugging
vpn_connected = False
for _ in range(30):  # Try for up to 30 seconds
    output = openvpn_process.stdout.readline().decode('utf-8')
    error_output = openvpn_process.stderr.readline().decode('utf-8')
    if "Initialization Sequence Completed" in output:
        vpn_connected = True
        break
    if error_output:
        print(f"Error: {error_output.strip()}")
    time.sleep(1)

if vpn_connected:
    print("VPN connection established successfully.")
else:
    print("Failed to establish VPN connection.")
    openvpn_process.terminate()
    raise RuntimeError("VPN connection failed")




### Establish Connection and Sample

In [ ]:
# Establish a connection to the database
try:
    connection = psycopg2.connect(
        dbname=dbname,
        user=username,
        password=password,
        host=host,
        port=port
    )
    cursor = connection.cursor()
    print("Connection to the database established successfully")

    # Execute a sample query
    cursor.execute("SELECT version();")
    db_version = cursor.fetchone()
    print(f"Database version: {db_version}")

    # Close the cursor and connection
    cursor.close()
    connection.close()
    print("Database connection closed")
except Exception as error:
    print(f"Error connecting to the database: {error}")


### Obtain Data Schema Function

In [5]:
import psycopg2
from psycopg2 import sql

def get_column_names_with_sample(schema_name, table_name, **db_params):
    try:
        # Database connection parameters
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()

        # SQL query to fetch column names and first row values
        query = sql.SQL("""
            SELECT column_name
            FROM information_schema.columns
            WHERE table_schema = %s AND table_name = %s
        """)

        cursor.execute(query, (schema_name, table_name))
        columns = [row[0] for row in cursor.fetchall()]

        # SQL query to fetch the first row of values
        query = sql.SQL("SELECT * FROM {}.{} LIMIT 1").format(
            sql.Identifier(schema_name),
            sql.Identifier(table_name)
        )

        cursor.execute(query)
        sample_row = cursor.fetchone()

        # Close the connection
        cursor.close()
        conn.close()

        # Combine columns with sample values
        if sample_row:
            column_samples = [f"Column: {columns[i]}, Sample Value: {sample_row[i]}" for i in range(len(columns))]
        else:
            column_samples = [f"Column: {column}, Sample Value: None" for column in columns]
        
        return "\n".join(column_samples)

    except Exception as e:
        return f"An error occurred: {e}"

# Example usage
schema_name = 'app'
table_name = 'house'

schema = get_column_names_with_sample(schema_name, table_name, **db_params)
#print(schema)


### Generate SQL Query

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=api_key)


#prompt = "Find a home that fits 8 people, close to disney and has a pool."
prompt = "Does house id 215 have a pool?"
#prompt = "Show me a house without a pool"
#prompt = "Does house id 215 have a pool? How many people it can fit?"


def generate_sql_query(prompt):

  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            #"text": prompt+schema
            "text" : base_prompt+' Only output the SQL Query and return a maximum of 10 rows.'
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": prompt
          }
        ]
      }
    ],
    temperature=1,
    max_tokens=512,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response

response = generate_sql_query(prompt)

print(response.choices[0].message.content)

### SQL to Response Converter

In [ ]:
import json
from sqlalchemy import create_engine, text

def clean_sql_query(text):
    return text.replace('```sql\n', '').replace('\n```', '').replace('"""sql\n', '').replace('\n"""', '')

# Assuming response.choices[0].message.content contains the SQL query
sql_query = clean_sql_query(response.choices[0].message.content)

# Create a connection string
connection_string = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

# Connect to the PostgreSQL database using SQLAlchemy
engine = create_engine(connection_string)
with engine.connect() as conn:
    # Execute the query and fetch the results
    result = conn.execute(text(sql_query))
    rows = result.fetchall()

# Convert the result to a list of dictionaries
data = [dict(row._asdict()) for row in rows]


# Add the URL to each result
for item in data:
    if 'id' in item:
        item['URL'] = f"goweasy.com/house-details/{item['id']}"
        break
else:
    pass

# Convert the data to a JSON response
json_response = json.dumps({"results": data})

#print(json_response)

# Example of how to use the JSON response
# This would be sent to the chatbot for interpretation
# chatbot_interpret(json_response)


### Prompt to SQL to Response

In [ ]:
from openai import OpenAI
import json
from sqlalchemy import create_engine, text


#prompt = "Find a home that fits 8 people, close to disney and has a pool."
prompt = "Does house id 215 have a pool?"
#prompt = "Show me a house without a pool"
#prompt = "Does house id 215 have a pool? How many people it can fit?"


def get_answer_from_prompt(prompt):
    
  client = OpenAI(api_key=api_key)
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text" : base_prompt+' Only output the SQL Query and return a maximum of 10 rows.'
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": prompt
          }
        ]
      }
    ],
    temperature=1,
    max_tokens=512,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )

  # Assuming response.choices[0].message.content contains the SQL query
  sql_query = response.choices[0].message.content.replace('```sql\n', '').replace('\n```', '').replace('"""sql\n', '').replace('\n"""', '')

  # Create a connection string
  connection_string = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

  # Connect to the PostgreSQL database using SQLAlchemy
  engine = create_engine(connection_string)
  with engine.connect() as conn:
      # Execute the query and fetch the results
      result = conn.execute(text(sql_query))
      rows = result.fetchall()

  # Convert the result to a list of dictionaries
  data = [dict(row._asdict()) for row in rows]


  # Add the URL to each result
  for item in data:
      if 'id' in item:
          item['URL'] = f"goweasy.com/house-details/{item['id']}"
          break
  else:
      pass

  # Convert the data to a JSON response
  json_response = json.dumps({"results": data})
  return json_response

#print(get_answer_from_prompt(prompt))

# Example of how to use the JSON response
# This would be sent to the chatbot for interpretation
# chatbot_interpret(json_response)


### Troubleshoot SQL to find specific house

In [ ]:
import json
from sqlalchemy import create_engine, text
from datetime import datetime

def clean_sql_query(text):
    return text.replace('```sql\n', '').replace('\n```', '').replace('"""sql\n', '').replace('\n"""', '')

# Define the house ID you want to fetch information for
house_id = 172

# Assuming response.choices[0].message.content contains the SQL query
# Adjust the query to fetch all information for the specific house ID
sql_query = f"SELECT * FROM app.house WHERE id = {house_id};"

# Create a connection string
connection_string = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

# Connect to the PostgreSQL database using SQLAlchemy
engine = create_engine(connection_string)
with engine.connect() as conn:
    # Execute the query and fetch the results
    result = conn.execute(text(sql_query))
    rows = result.fetchall()

# Convert the result to a list of dictionaries
data = []
for row in rows:
    row_dict = dict(row._asdict())
    for key, value in row_dict.items():
        if isinstance(value, datetime):
            row_dict[key] = value.isoformat()  # Convert datetime to string in ISO 8601 format
    data.append(row_dict)

# Convert the data to a JSON response
json_response = json.dumps({"results": data})

print(json_response)


#json.loads(json_response)["results"][0]["have_pool"]


### Database Project Code Main

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

def clean_sql_query(text):
    return text.replace('```sql\n', '').replace('\n```', '').replace('"""sql\n', '').replace('\n"""', '')


sql_query = clean_sql_query(response.choices[0].message.content)

# Create a connection string
connection_string = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

# Connect to the PostgreSQL database using SQLAlchemy
engine = create_engine(connection_string)
with engine.connect() as conn:
    # Execute the query and fetch the results
    df = pd.read_sql_query(sql_query, conn)

# Add a new column for the URL
df['URL'] = df['id'].apply(lambda x: f"goweasy.com/house-details/{x}")

# Export the results to an Excel file
output_file = 'homes_near_disney_with_pool.xlsx'
df.to_excel(output_file, index=False)

print(f"Data exported to {output_file}")



def get_answer_from_prompt(user_prompt):
    """
    Takes a user prompt, converts it into an SQL query, executes the query on the database, and returns the results.

    This function interprets a natural language prompt from the user, generates an appropriate SQL query,
    executes the query on the connected database, and returns the query results. It handles a variety of 
    questions related to the database, such as retrieving specific records, filtering data based on conditions,
    and aggregating data.

    Parameters:
        user_prompt (str): The natural language prompt provided by the user.

    Returns:
        list of dict: A list of dictionaries, each representing a row in the query result.
    """
    # Function implementation goes here
